In [1]:
!pip install fastapi
!pip install uvicorn
!pip install python-dotenv
!pip install sqlalchemy
!pip install cachetools
!pip install pandas

In [3]:
import sqlite3
import pandas as pd
from pathlib import Path
import os
from IPython.display import display, HTML

In [4]:
class DatabaseViewer:
    def __init__(self, db_path):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
    
    def get_tables(self):
        self.cursor.execute("""
            SELECT name 
            FROM sqlite_master 
            WHERE type='table' 
            AND name NOT LIKE 'sqlite_%';
        """)
        return [table[0] for table in self.cursor.fetchall()]
    
    def get_table_info(self, table_name):
        self.cursor.execute(f"PRAGMA table_info({table_name});")
        columns = self.cursor.fetchall()
        self.cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
        row_count = self.cursor.fetchone()[0]
        
        return {
            'columns': pd.DataFrame(columns, columns=['cid', 'name', 'type', 'notnull', 'dflt_value', 'pk']),
            'row_count': row_count
        }
    
    def query_table(self, table_name, limit=5):
        try:
            return pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT {limit}", self.conn)
        except Exception as e:
            return f"Error querying table {table_name}: {str(e)}"
    
    def display_database_info(self):
        tables = self.get_tables()
        print(f"📊 Total Tables: {len(tables)}\n")
        
        for table in tables:
            info = self.get_table_info(table)
            print(f"\n🗃️ Table: {table}")
            print(f"   Rows: {info['row_count']}")
            print("\n📋 Schema:")
            display(info['columns'])
            print("\n📝 Sample Data:")
            display(self.query_table(table))
            print("-" * 80)
    
    def execute_custom_query(self, query):
        try:
            return pd.read_sql_query(query, self.conn)
        except Exception as e:
            return f"Query error: {str(e)}"
    
    def execute_update_query(self, query):
        try:
            self.cursor.execute(query)
            self.conn.commit()  # Importante: commit los cambios
            return "Update executed successfully"
        except Exception as e:
            self.conn.rollback()  # En caso de error, hacer rollback
            return f"Update error: {str(e)}"
    
    def close(self):
        self.conn.close()

In [5]:
db_path = "../../chat-Interface/db.sqlite3"
viewer = DatabaseViewer(db_path)

In [6]:
viewer.display_database_info()

📊 Total Tables: 17


🗃️ Table: django_migrations
   Rows: 23

📋 Schema:


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,1,None,1
1,1,app,varchar(255),1,None,0
2,2,name,varchar(255),1,None,0
3,3,applied,datetime,1,None,0



📝 Sample Data:


,id,app,name,applied
0,1,contenttypes,0001_initial,2024-09-26 21:20:14.671169
1,2,auth,0001_initial,2024-09-26 21:20:14.725782
2,3,admin,0001_initial,2024-09-26 21:20:14.778684
3,4,admin,0002_logentry_remove_auto_add,2024-09-26 21:20:14.820878
4,5,admin,0003_logentry_add_action_flag_choices,2024-09-26 21:20:14.875153


--------------------------------------------------------------------------------

🗃️ Table: auth_group_permissions
   Rows: 0

📋 Schema:


...

--------------------------------------------------------------------------------

🗃️ Table: chat_property
   Rows: 9

📋 Schema:


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,1,None,1
1,1,location,varchar(255),1,None,0
2,2,price,decimal,1,None,0
3,3,square_meters,decimal,1,None,0
4,4,property_type,varchar(20),1,None,0
5,5,description,TEXT,0,None,0
6,6,created_at,datetime,1,None,0
7,7,num_bedrooms,INTEGER,1,None,0
8,8,num_rooms,INTEGER,1,None,0
9,9,project_category,varchar(20),1,None,0



📝 Sample Data:


,id,location,price,square_meters,property_type,description,created_at,num_bedrooms,num_rooms,project_category,project_type,residence_type,city_id,country_id,province_id,image,url
0,18,"Palermo, Capital Federal",225000,150.00,apartment,Amplio y Luminoso PH Con gran terraza en excel...,2024-12-10 15:07:02.523933,6,8,premium,pozo,familiar,None,2,None,property_images/casa2.jpeg,https://cabaprop.com.ar/propiedad/67584b50fef2...
1,19,"Villa Urquiza, Capital Federal",137000,53.00,apartment,Vendo en VILLA URQUIZA\r\n\r\n*APTO CREDITO*\r...,2024-12-10 15:09:52.913019,2,4,premium,no_pozo,unipersonal,None,2,None,property_images/apto2.jpeg,https://cabaprop.com.ar/propiedad/64d0a347d7bb...
2,20,"Envigado, Antioquía",370000,73.99,apartment,"Vivir en Civita es vivir urbano, es sentir la ...",2024-12-10 15:30:53.700602,3,6,social,pozo,familiar,None,3,None,,https://www.estrenarvivienda.com/civita/47125
3,21,"Armenia, Quindio",54000,43.52,apartment,Los apartamentos de Solterra cuentan con sala ...,2024-12-10 15:32:42.704242,1,2,premium,pozo,unipersonal,None,3,None,property_images/col2.jpg,https://www.estrenarvivienda.com/solterra/47301
4,22,"La Planicie 3era Etapa, La Molina, Lima",365000,285.00,house,Casa de 3 pisos con 3 terrazas y excelentes ac...,2024-12-10 15:35:49.272373,3,9,luxury,pozo,familiar,None,4,None,property_images/pe1.jpeg,https://babilonia.pe/inmueble/casa-en-venta-en...


--------------------------------------------------------------------------------


In [7]:
# Consultas personalizadas
print('\nConsulta personalizada:')
query = """
SELECT name, sql 
FROM sqlite_master 
WHERE type='table' 
AND name NOT LIKE 'sqlite_%';
"""
display(viewer.execute_custom_query(query))

# Cerrar conexión
# viewer.close()


Consulta personalizada:


,name,sql
0,django_migrations,"CREATE TABLE ""django_migrations"" (""id"" integer..."
1,auth_group_permissions,"CREATE TABLE ""auth_group_permissions"" (""id"" in..."
2,auth_user_groups,"CREATE TABLE ""auth_user_groups"" (""id"" integer ..."
3,auth_user_user_permissions,"CREATE TABLE ""auth_user_user_permissions"" (""id..."
4,django_admin_log,"CREATE TABLE ""django_admin_log"" (""id"" integer ..."
5,django_content_type,"CREATE TABLE ""django_content_type"" (""id"" integ..."
6,auth_permission,"CREATE TABLE ""auth_permission"" (""id"" integer N..."
7,auth_group,"CREATE TABLE ""auth_group"" (""id"" integer NOT NU..."
8,auth_user,"CREATE TABLE ""auth_user"" (""id"" integer NOT NUL..."
9,chat_chunk,"CREATE TABLE ""chat_chunk"" (""id"" integer NOT NU..."


In [13]:
# 1. Análisis por País
query_by_country = """
SELECT country_id, 
       COUNT(*) as total_properties,
       AVG(price) as avg_price,
       AVG(square_meters) as avg_size
FROM chat_property
GROUP BY country_id
ORDER BY total_properties DESC;
"""

# 2. Análisis por tipo de propiedad
query_price_by_type = """
SELECT property_type,
       COUNT(*) as count,
       AVG(price) as avg_price,
       MIN(price) as min_price,
       MAX(price) as max_price
FROM chat_property
GROUP BY property_type
ORDER BY avg_price DESC;
"""

# 3. Tendencia temporal
query_price_trend = """
SELECT strftime('%Y-%m', created_at) as month,
       COUNT(*) as listings,
       AVG(price) as avg_price,
       AVG(square_meters) as avg_size
FROM chat_property
GROUP BY month
ORDER BY month;
"""

# 4. Análisis por ubicación
query_location_stats = """
SELECT province_id, city_id,
       COUNT(*) as total_properties,
       AVG(price) as avg_price,
       AVG(price/square_meters) as price_per_sqm
FROM chat_property
GROUP BY province_id, city_id
ORDER BY avg_price DESC;
"""

# 5. Distribución por tipo
query_property_distribution = """
SELECT project_type,
       residence_type,
       COUNT(*) as count,
       AVG(price) as avg_price,
       AVG(square_meters) as avg_size
FROM chat_property
GROUP BY project_type, residence_type
ORDER BY count DESC;
"""

# 6. Análisis de habitaciones
query_rooms_analysis = """
SELECT num_bedrooms,
       COUNT(*) as count,
       AVG(price) as avg_price,
       AVG(square_meters) as avg_size,
       AVG(price/square_meters) as price_per_sqm
FROM chat_property
GROUP BY num_bedrooms
ORDER BY num_bedrooms;
"""

In [14]:

def run_analysis(viewer):
    queries = {
        "Análisis por País": query_by_country,
        "Análisis por Tipo": query_price_by_type,
        "Tendencia Temporal": query_price_trend,
        "Análisis por Ubicación": query_location_stats,
        "Distribución por Tipo": query_property_distribution,
        "Análisis de Habitaciones": query_rooms_analysis
    }
    
    for title, query in queries.items():
        print(f"\n=== {title} ===")
        result = viewer.execute_custom_query(query)
        display(result)
        print("-"*80)


In [15]:

# Uso:
run_analysis(viewer)


=== Análisis por País ===


,country_id,total_properties,avg_price,avg_size
0,None,6,265000.0,158.833333


--------------------------------------------------------------------------------

=== Análisis por Tipo ===


,property_type,count,avg_price,min_price,max_price
0,apartment,5,298000.0,50000,500000
1,cabin,1,100000.0,100000,100000


--------------------------------------------------------------------------------

=== Tendencia Temporal ===


...

--------------------------------------------------------------------------------

=== Distribución por Tipo ===


,project_type,residence_type,count,avg_price,avg_size
0,1,1,5,308000.0,170.6
1,pozo,familiar,1,50000.0,100.0


--------------------------------------------------------------------------------

=== Análisis de Habitaciones ===


,num_bedrooms,count,avg_price,avg_size,price_per_sqm
0,1,6,265000.0,158.833333,1965.666667


--------------------------------------------------------------------------------


In [12]:
# Consulta para ver todas las tablas
query = """
SELECT name 
FROM sqlite_master 
WHERE type='table' 
AND name NOT LIKE 'sqlite_%';
"""

# Ver la estructura de las tablas
print("Tablas en la base de datos:")
display(viewer.execute_custom_query(query))

Tablas en la base de datos:


,name
0,django_migrations
1,auth_group_permissions
2,auth_user_groups
3,auth_user_user_permissions
4,django_admin_log
5,django_content_type
6,auth_permission
7,auth_group
8,auth_user
9,chat_chunk


In [7]:
# Consulta para ver todas las tablas
query = """
SELECT cp.*,
                        ROUND(cp.price, 2) as price,
                        cc.name as country_name,
                        cd.name as province_name,
                        cct.name as city_name
                    FROM chat_property cp
                    LEFT JOIN chat_country cc ON cp.country_id = cc.id
                    LEFT JOIN chat_province cd ON cp.province_id = cd.id
                    LEFT JOIN chat_city cct ON cp.city_id = cct.id
                    ;
"""

# Ver la estructura de las tablas
print("Tablas en la base de datos:")
display(viewer.execute_custom_query(query))

Tablas en la base de datos:


,id,location,price,square_meters,property_type,description,created_at,num_bedrooms,num_rooms,project_category,...,residence_type,city_id,country_id,province_id,image,url,price,country_name,province_name,city_name
0,18,"Palermo, Capital Federal",225000,150.00,apartment,Amplio y Luminoso PH Con gran terraza en excel...,2024-12-10 15:07:02.523933,6,8,premium,...,familiar,None,2,None,property_images/casa2.jpeg,https://cabaprop.com.ar/propiedad/67584b50fef2...,225000.0,Argentina,None,None
1,19,"Villa Urquiza, Capital Federal",137000,53.00,apartment,Vendo en VILLA URQUIZA\r\n\r\n*APTO CREDITO*\r...,2024-12-10 15:09:52.913019,2,4,premium,...,unipersonal,None,2,None,property_images/apto2.jpeg,https://cabaprop.com.ar/propiedad/64d0a347d7bb...,137000.0,Argentina,None,None
2,20,"Envigado, Antioquía",370000,73.99,apartment,"Vivir en Civita es vivir urbano, es sentir la ...",2024-12-10 15:30:53.700602,3,6,social,...,familiar,None,3,None,,https://www.estrenarvivienda.com/civita/47125,370000.0,Colombia,None,None
3,21,"Armenia, Quindio",54000,43.52,apartment,Los apartamentos de Solterra cuentan con sala ...,2024-12-10 15:32:42.704242,1,2,premium,...,unipersonal,None,3,None,property_images/col2.jpg,https://www.estrenarvivienda.com/solterra/47301,54000.0,Colombia,None,None
4,22,"La Planicie 3era Etapa, La Molina, Lima",365000,285.00,house,Casa de 3 pisos con 3 terrazas y excelentes ac...,2024-12-10 15:35:49.272373,3,9,luxury,...,familiar,None,4,None,property_images/pe1.jpeg,https://babilonia.pe/inmueble/casa-en-venta-en...,365000.0,Peru,None,None
5,23,"Alt. Del Cruce De Acapulco Con Rapallo, La Mol...",555000,404.00,house,Casa de dos pisos rodeada de áreas verdes 404 ...,2024-12-10 15:39:16.732483,3,10,social,...,familiar,None,4,None,property_images/pe3.jpeg,https://babilonia.pe/inmueble/casa-en-venta-en...,555000.0,Peru,None,None
6,24,"Los Alamos De Monterrico, Santiago De Surco, Lima",1200000,6000.00,house,La casa más grande en Los Álamos de Monterrico...,2024-12-10 15:46:25.012200,5,17,luxury,...,familiar,None,4,None,property_images/pe4.jpeg,https://babilonia.pe/inmueble/casa-en-venta-en...,1200000.0,Peru,None,None
7,25,"Santa Paula, Caracas",78000,104.10,apartment,Acogedor apartamento en venta en Santa Paula e...,2024-12-10 15:51:20.172633,3,6,social,...,familiar,None,5,None,property_images/ve1.jpg,https://mercadopiso.com/property/apartamento-s...,78000.0,Venezuela,None,None
8,26,"La Boyera, Caracas",180000,570.00,house,Amplia casa en excelente urbanizacion con call...,2024-12-10 15:53:45.506500,5,15,social,...,familiar,None,5,None,property_images/ve2.jpg,https://mercadopiso.com/property/casa-la-boyer...,180000.0,Venezuela,None,None


In [1]:
class DatabaseViewer:
    def __init__(self, db_path):
        self.db_path = db_path
        self.conn = None
        self.cursor = None
    
    def connect(self):
        if not self.conn:
            self.conn = sqlite3.connect(self.db_path)
            self.cursor = self.conn.cursor()
    
    def execute_update_query(self, query):
        self.connect()  # Aseguramos que hay una conexión activa
        try:
            self.cursor.execute(query)
            self.conn.commit()
            return "Update executed successfully"
        except Exception as e:
            if self.conn:
                self.conn.rollback()
            return f"Update error: {str(e)}"
    
    def execute_custom_query(self, query):
        self.connect()  # Aseguramos que hay una conexión activa
        try:
            return pd.read_sql_query(query, self.conn)
        except Exception as e:
            return f"Query error: {str(e)}"
    
    def close(self):
        if self.conn:
            self.conn.close()
            self.conn = None
            self.cursor = None

# Uso
db_path = "../../chat-Interface/db.sqlite3"
viewer = DatabaseViewer(db_path)


In [4]:
# 	id	name	country_id
# 0	5	Rosario	5
# 1	6	Lima	4
# 2	7	Quindio	3
# 3	8	Antioquía	3
# 4	9	Capital Federal	2


# 	id	name	province_id
# 0	1	Recoleta	9
# 1	2	Palermo	9
# 2	3	Villa Urquiza	9
# 3	4	Envigado	8
# 4	5	Armenia	7

# 	id	name
# 0	1	Ciudad Autonoma de Buenos Aires
# 1	2	Argentina
# 2	3	Colombia
# 3	4	Peru
# 4	5	Venezuela

update_query = """

UPDATE chat_property
SET province_id = 8, city_id = 4
WHERE id = 20;
"""

#-----------------------------------

# UPDATE chat_property
# SET province_id = 9, city_id = 1
# WHERE id = 17;
# """

# UPDATE chat_property
# SET province_id = 9, city_id = 2
# WHERE id = 18;
# """

# UPDATE chat_property
# SET province_id = 9, city_id = 3
# WHERE id = 19;
# """



# UPDATE chat_property
# SET province_id = 7, city_id = 5
# WHERE id = 21;
# """

# UPDATE chat_property
# SET province_id = 6, city_id = 6
# WHERE id = 22;
# """

# UPDATE chat_property
# SET province_id = 6, city_id = 6
# WHERE id = 23;
# """

# UPDATE chat_property
# SET province_id = 6, city_id = 7
# WHERE id = 24;
# """

# UPDATE chat_property
# SET province_id = 10, city_id = 8
# WHERE id = 25;
# """

# UPDATE chat_property
# SET province_id = 10, city_id = 9
# WHERE id = 26;
# """

# Ver la estructura de las tablas
try:
    # Ejecutar el update
    result = viewer.execute_update_query(update_query)
    print(result)
    
    # # Verificar el resultado
    # verify_query = """
    # SELECT id, province_id, city_id 
    # FROM chat_property 
    # WHERE id = 17;
    # """
    # print("\nVerificación del update:")
    # display(viewer.execute_custom_query(verify_query))

finally:
    viewer.close()

Update executed successfully
